In [24]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.nodes import *
from src.mcmc import *

def get_stats(district_type):
    nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract10'
#     nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}'
    M = MCMC(random_seed=9000000, nodes_tbl=nodes_tbl)
    M.post_process1()
    M.post_process2()
    query = f"select * from {M.hash_tbl}"
    df = run_query(query)
    if district_type == 'cd':
        pop_dev = 1
        base_defect = 22
    elif district_type == 'sldu':
        pop_dev = 10
        base_defect = 13
    elif district_type == 'sldl':
        pop_dev = 10
        base_defect = 19
    loose_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect*3)
    tight_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect)
    print(f'\n\n{district_type} loose mask stats')
    display(df.loc[loose_mask, 'defect_plan'].describe())
    print(f'{district_type} tight mask stats')
    display(df.loc[tight_mask, 'defect_plan'].describe())
    return df

df_cd   = get_stats('cd')
df_sldl = get_stats('sldu')
df_sldu = get_stats('sldl')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021
Stacking summaries & uniquifying ... 

cd loose mask stats


count    79629.000000
mean        30.773035
std          5.985842
min         19.000000
25%         23.000000
50%         33.000000
75%         35.000000
max         46.000000
Name: defect_plan, dtype: float64

cd tight mask stats


count    11918.000000
mean        21.885300
std          0.359272
min         19.000000
25%         22.000000
50%         22.000000
75%         22.000000
max         22.000000
Name: defect_plan, dtype: float64

Stacking summaries & uniquifying ... 

sldu loose mask stats


...

count    24724.000000
mean        10.702637
std          1.513223
min          7.000000
25%          9.000000
50%         11.000000
75%         12.000000
max         13.000000
Name: defect_plan, dtype: float64

Stacking summaries & uniquifying ... 

sldl loose mask stats


count    339176.000000
mean         41.868543
std           7.292072
min          14.000000
25%          37.000000
50%          42.000000
75%          47.000000
max          57.000000
Name: defect_plan, dtype: float64

sldl tight mask stats


count    6164.000000
mean       17.880435
std         1.208902
min        14.000000
25%        17.000000
50%        18.000000
75%        19.000000
max        19.000000
Name: defect_plan, dtype: float64

In [ ]:
query = f"""
    select
        seed,
        plan,
        cd,
        substring(geoid, 1, 4) as cnty,
    from
        {results_old} as A
    left join (
        select
            substring(geoid, 3) as geoid,
            substring(cnty, 3) as cnty
        from
            {assignments}
        ) as B
    on
        A.geoid = B.geoid
    limit 100
"""


In [ ]:
query = f"""
            select
                substring(geoid, 1, 3) as cnty,
                sum({seats_col}) as {seats_col}
            from
                {nodes}
"""



In [ ]:
nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_tabblock_{district_type}_contract0'

query = f"""


"""

In [103]:
pop = 29145505
n = 38
target_pop = pop / n
district_type = 'cd'
seats_col = f'seats_{district_type}'
results_old = f'cmat-315920.TX_2020_cntyvtd_cd.TX_2020_cntyvtd_{district_type}_0000000_allresults_stack'
nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract2010'
output = f'cmat-315920.redistricting_data.contract_compares'

query = f"""
select
    *,
    abs(wholes_defect) + abs(intersects_defect) as defect
from (
    select
        *,
        floor({seats_col}) - wholes as wholes_defect,
        intersects - ceiling({seats_col}) as intersects_defect
    from (
        select
            seed,
            plan,
            cnty,
            max(pop_deviation) as pop_deviation,
            sum({seats_col}) as {seats_col},
            sum(whole) as wholes,
            count(*) as intersects
        from (
            select
                *,
                case when count(distinct cnty) over (partition by seed, plan, cd) = 1 then 1 else 0 end as whole
            from (
                select
                    A.seed,
                    A.plan,
                    A.cnty,
                    A.cd,
                    max(A.pop_deviation) as pop_deviation,
                    sum(B.{seats_col}) as {seats_col}
                from (
                    select
                        seed,
                        plan,
                        substring(geoid, 1, 3) as cnty,
                        cd,
                        geoid,
                        pop_imbalance_plan as pop_deviation
                    from
                        {results_old}
                    ) as A
                inner join
                    {nodes} as B
                on
                    A.geoid = B.geoid
                group by
                    seed, plan, cnty, cd
                )
            )
        group by
            seed, plan, cnty
        )
    )
"""
df = load_table(output, query=query)

In [92]:
df.sort_values('whole')

,seed,plan,cnty,pop_deviation,seats_cd,whole,intersects
0,1126,552,085,7.060746,1.387853,0,5
160,1126,552,443,7.060746,0.000991,0,1
161,1126,552,035,7.060746,0.023775,0,1
162,1126,552,031,7.060746,0.014829,0,1
163,1126,552,221,7.060746,0.080312,0,1
...,...,...,...,...,...,...,...
125,1126,552,141,7.060746,1.128646,1,2
15,1126,552,453,7.060746,1.682151,1,5
187,1126,552,439,7.060746,2.751859,1,5
41,1126,552,113,7.060746,3.407540,2,7


In [86]:
df[seats_col].sum()

38.00000000000003

In [55]:
pop = 29145505
n = 38
target_pop = pop / n
results_old = 'cmat-315920.TX_2020_cntyvtd_cd.TX_2020_cntyvtd_cd_0000000_allresults_stack'
assignments = 'cmat-315920.redistricting_data.assignments_TX_2020_tabblock'

query = f"""
select
    seed,
    plan,
    cnty,
    max(pop_deviation) as pop_deviation,
    sum(whole) as whole
from (    
    select
        *,
        case when count(distinct cnty) over (partition by seed, plan, cd) = 1 then 1 else 0 end as whole
    from (
        select
            seed,
            plan,
            cnty,
            cd,
            max(pop_deviation) as pop_deviation
        from (
            select
                seed,
                plan,
                substring(geoid, 1, 3) as cnty,
                cd,
                geoid,
                pop_imbalance_plan as pop_deviation
            from
                {results_old}
            where
                seed = 1126 and plan = 552
            )
        group by
            seed, plan, cnty, cd
        )
    )
group by
    seed, plan, cnty
"""

df = run_query(query)
#     count(*) over (partition byhash_plan,
df

,seed,plan,cnty,pop_deviation,whole
0,1126,552,121,7.060746,0
1,1126,552,183,7.060746,0
2,1126,552,423,7.060746,0
3,1126,552,321,7.060746,0
4,1126,552,157,7.060746,0
...,...,...,...,...,...
249,1126,552,103,7.060746,0
250,1126,552,019,7.060746,0
251,1126,552,457,7.060746,0
252,1126,552,277,7.060746,0


In [26]:
f = f'nodes_TX_2020_tabblock_sldu_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)

In [27]:
f = f'nodes_TX_2020_tabblock_sldl_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)